In [18]:

import arviz as az
import pymc as py
import pandas as pd
import pandas as np
from datafunction import *
import aesara.tensor as at


In [2]:
TGW_df = pd.read_csv('TransformedData/GWF.csv')
TGW_df.sample(5)

,Unnamed: 0,name,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,GW_step,kickoff_time_t_step,Tally_Points_step,was_home_ind,was_home_step_ind,ind_DEF,ind_FWD,ind_GK,ind_MID,playtime
27340,1102196,Harry Tyrer,Everton,0.5,0,0,0,0,0.0,616,...,23,2022-01-22T12:30:00Z,19,0,1,0,0,1,0,0
22348,904761,Jordan Pickford,Everton,4.0,0,0,17,0,0.0,170,...,9,2021-10-23T14:00:00Z,14,1,1,0,0,1,0,1
27064,1092086,Louie Moulden,Wolves,0.0,0,0,0,0,0.0,611,...,30,2022-03-18T20:00:00Z,46,0,1,0,0,1,0,0
25430,1030342,Robert Elliot,Watford,-1.5,0,0,0,0,0.0,574,...,35,2022-04-30T14:00:00Z,22,0,1,0,0,1,0,0
6360,261974,Robin Koch,Leeds,0.0,0,0,0,0,0.0,194,...,11,2021-11-07T14:00:00Z,11,0,1,1,0,0,0,0


In [3]:
# Isolate what we need from this dependant variables

depend = TGW_df[[
            'GW'                                                    # Gameweek
            ,'playtime'                                             # this is an indicator function
            ,'xP'                                                   # this is an XP approach, i actually don't know what it means
            ,'transfers_balance', 'transfers_in', 'transfers_out'   # have to standardize this
            ,'ind_DEF', 'ind_FWD','ind_MID'                         #Position of players
            ,'was_home_ind'                                         # currently was this a home game?
            ,'was_home_step_ind'                                    # temporily this information is available to use
            ,'value'                                                # Current value of the player
            ,'total_points'                                         # Current performance of the player
            ,'Tally_Points'                                         # Points collected so far, need to rework this
            ,'value'                                                # Value of player
            ,'total_points_step'                                    #
            ]]



In [13]:
def total_point_correction(row):
    # for row wise computation
    '''
    To correct issue with -ve points which is not condusive to my model
    '''

    if row <-0:
        result = 0
    else:
        result = row
    return result

explain = depend['total_points_step'].apply(total_point_correction)


In [20]:
with py.Model() as TP_count:

    #define priors as weekly 
    b0 = py.Normal("intercept", mu=0, sigma=10)
    b1 = py.Normal("value", mu= 40,sigma=10)            #based on a quick spot check of graphs.ipynb
    b2 = py.Normal("transfers_out", mu= 0,sigma=10)
    b3 = py.Normal("transfers_in", mu= 0,sigma=10)

    # our linear model 
    # theta = b0 +b1 * T1['value'] + b2 *T1['transfers_out'] + b3 *T1['transfers_in']
    theta = at.dscalar('theta')
    theta = b0 + (b1 * depend['value'].values) + (b2 * depend['transfers_out'].values) + (b3 * depend['transfers_in'].values )  
    print((theta.ndim))

    # define poisson likelihood
    # y = py.Poisson("y", mu=py.math.exp(theta), observed = (explain.values  ))  # one step ahead dependency forward

2


In [21]:
theta = at.dscalar('theta') 
theta = b0 + (b1 * depend['value'].values) + (b2 * depend['transfers_out'].values) + (b3 * depend['transfers_in'].values )  

In [22]:
theta

Elemwise{add,no_inplace}.0